In [11]:
import numpy as np
from grpc.beta import implementations
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2
import tensorflow as tf


from tensorflow_serving.apis import prediction_service_pb2_grpc


In [2]:
# https://weiminwang.blog/2017/09/12/introductory-guide-to-tensorflow-serving/
# https://medium.com/epigramai/tensorflow-serving-101-pt-2-682eaf7469e7

In [3]:
def get_stub(host='127.0.0.1', port='8500'):
    channel = implementations.insecure_channel(host, int(port))
    stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)
    return stub

In [4]:
def get_model_prediction(stub, model_name='model', signature_name='serving_default'):

    request = predict_pb2.PredictRequest()
    request.model_spec.name = model_name
    request.model_spec.signature_name = signature_name
    
    data = np.random.randint(100000, size=(100, 20)).astype(np.float32)

    #request.inputs['input'].CopyFrom(tf.contrib.util.make_tensor_proto(data, shape=data.shape))
    
    request.inputs['input'].CopyFrom(tf.contrib.util.make_tensor_proto(data, verify_shape=True, shape=(100, 20)))
 
    response = stub.Predict.future(request, 5.0)  # wait max 5s
    print(response.result())
    return response.result().outputs["output"].float_val

In [12]:
stub = get_stub(host='http://192.168.64.13', port='30125')

/usr/local/lib/python3.5/dist-packages/tensorflow_serving/apis/prediction_service_pb2.py:131: DeprecationWarning: beta_create_PredictionService_stub() method is deprecated. This method will be removed in near future versions of TF Serving. Please switch to GA gRPC API in prediction_service_pb2_grpc.
  'prediction_service_pb2_grpc.', DeprecationWarning)


In [10]:
get_model_prediction(stub)

_Rendezvous: <_Rendezvous of RPC that terminated with (StatusCode.UNAVAILABLE, Name resolution failure)>